In [7]:
train.data <- read.csv(file.path("..", "data", "training_data.csv"))
test.data <- read.csv(file.path("..", "data", "test_data.csv"))
sampleSubmission <- read.csv(file.path("..", "data", "sampleSubmission.csv"))

In [8]:
#delets dependent variables
train.dub <- train.data[!duplicated(as.list(train.data))]


In [9]:
#data set with VALENCE.PLEASANTNESS + V.. (with low varinace)
length <- length(train.dub[1,])
idx.low.var <- apply(train.dub[,4:length], 2, var) < 0.24 # identify pixels with low variance 
V.data <- data.frame(train.dub[,4:length])
V.data <- data.frame(V.data[,!idx.low.var])
V.data$VALENCE.PLEASANTNESS <- train.data$VALENCE.PLEASANTNESS

#split the data set into a training set and a validation set
dim <- length(V.data[,1])
idx.validation <- sample(1:dim, dim/3)

train <- V.data[-idx.validation,]
validation <- V.data[idx.validation,]



In [11]:
library(leaps)

In [12]:
#forward selection 
nvmax = 50
models.forward <- regsubsets(VALENCE.PLEASANTNESS ~ ., data = train, nvmax = nvmax, method = 'forward')


In [13]:
predict.regsubsets <- function(object,newdata,id,...){
form <- as.formula(object$call[[2]])
mat <- model.matrix(form,newdata)
coefi <- coef(object,id=id)
xvars <- names(coefi)
mat[,xvars]%*%coefi
}



In [15]:
#MSE of all selected models
for (i in 1:nvmax){
    p <- predict.regsubsets(models.forward, validation, id = i)
    MSE <- mean((p - validation$VALENCE.PLEASANTNESS)^2)
    print(MSE)
}


[1] 571.5789
[1] 549.7127
[1] 550.9349
[1] 560.389
[1] 553.9221
[1] 548.5557
[1] 533.4722
[1] 538.0865
[1] 534.7593
[1] 537.2705
[1] 535.263
[1] 541.1595
[1] 554.3138
[1] 557.5567
[1] 564.0819
[1] 559.2748
[1] 559.3812
[1] 562.7809
[1] 566.1274
[1] 571.3053
[1] 582.3196
[1] 578.3625
[1] 575.2865
[1] 570.1754
[1] 567.9635
[1] 568.7708
[1] 563.3812
[1] 561.9431
[1] 562.0193
[1] 561.9961
[1] 564.865
[1] 567.8246
[1] 568.1862
[1] 568.5194
[1] 571.3947
[1] 566.8093
[1] 562.3213
[1] 566.0267
[1] 562.7268
[1] 562.4681
[1] 563.6276
[1] 560.4724
[1] 559.6155
[1] 562.8109
[1] 564.9638
[1] 567.8904
[1] 567.1246
[1] 569.4156
[1] 566.7771
[1] 566.0377
